In [1]:
import pandas as pd
import requests
import json 
import numpy as np
import matplotlib.pyplot as pl

In [2]:
import os
import pickle
import geopandas as gpd
from shapely.geometry import Polygon, Point

In [3]:
from sklearn import svm